In [2]:
cd Internship/CEEW/

d:\Internship\CEEW


In [5]:
!py -m pip install PyPDF2


[notice] A new release of pip is available: 23.3.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import requests
import io
import re
import PyPDF2
import os

In [3]:
# Define the path for saving PDFs and the output CSV
output_folder = "Jobpdfs"
os.makedirs(output_folder, exist_ok=True)
output_csv_path = os.path.join(output_folder, "extracted_job_info.csv")

In [4]:
# Load the dataset
df = pd.read_excel("D:\\Internship\\CEEW\\TM - QP list 30Apr24.xlsx")

# Define the list of sectors to keep
sectors_to_keep = ['Automotive', 'Food Processing', 'Green Jobs',
                   'Handicrafts and Carpet', 'Iron and Steel', 'Mining',
                   'Textile', 'Tourism & Hospitality']

# Filter the DataFrame to keep only the specified sectors
filtered_df = df[df['Sector'].isin(sectors_to_keep)]

In [5]:
filtered_df

,S No.,Sector,Sub Sector,Occupation,QP/Job Role Name,Unnamed: 5,QP/Job Role Code,Version,Last Reviewed On(DD/MM/YYYY),Next Review Date(DD/MM/YYYY),...,Common Norms Category,Manufacturing/Services/Agriculture,Deactivation Date(DD/MM/YYYY),Technical/Non Technical,Organised Or Unorganised Or Both,NQR code,QP Type,Minimum Education Qualification & Experience,Minimum Job Entry Age,Link
249,250,Automotive,Automotive Vehicle Sales (Dealer),Vehicle Sales,Automotive Sales Executive,Automotive_Automotive Sales Executive,ASC/Q1001,2.0,25/11/2021,25/11/2026,...,II,Services,NaN,Non-Technical,Both,2021/AUT/ASDC/04685,NaN,12th Class and 2 Years Experience - of releva...,18,https://s3.ap-south-1.amazonaws.com/nsdcproddo...
250,251,Automotive,Automotive Vehicle Sales (Dealer),Vehicle Sales,Automotive Sales Executive,Automotive_Automotive Sales Executive,ASC/Q1001,3.0,NaN,25/11/2025,...,II,Services,NaN,Non-Technical,Organized,2021/AUT/ASDC/04685,Unique,"12th Class,10th Class - + 2 years ITI,Certific...",18,https://s3.ap-south-1.amazonaws.com/nsdcproddo...
251,252,Automotive,Automotive Vehicle Sales (Dealer),Vehicle Sales,Automotive Sales Executive,Automotive_Automotive Sales Executive,ASC/Q1001,4.0,NaN,25/11/2025,...,II,Services,NaN,Non-Technical,Organized,2021/AUT/ASDC/04685,Unique,"10th Class - + 1 year ITI,10th Class - pass an...",18,https://s3.ap-south-1.amazonaws.com/nsdcproddo...
252,253,Automotive,Automotive Vehicle Sales (Dealer),Vehicle Sales,Automotive Sales Executive,Automotive_Automotive Sales Executive,ASC/Q1001,5.0,NaN,17/11/2025,...,II,Services,NaN,Non-Technical,Organized,2022/AUT/ASDC/06573,Unique,"10th Class - + 1 year ITI,10th Class - pass an...",18,https://s3.ap-south-1.amazonaws.com/nsdcproddo...
253,254,Automotive,Automotive Vehicle Sales (Dealer),Vehicle Sales,Automotive Sales Consultant,Automotive_Automotive Sales Consultant,ASC/Q1005,2.0,18/08/2021,31/08/2024,...,II,Services,NaN,Non-Technical,Both,2021/AUT/ASDC/04434,NaN,Graduate - in any discipline and 1 Year Experi...,21,https://s3.ap-south-1.amazonaws.com/nsdcproddo...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2725,2726,Textile,Handloom & Khadi,Spinner,Hand Spinning Operator,Textile_Hand Spinning Operator,TSC/Q7901,3.0,NaN,25/11/2024,...,II,Manufacturing,NaN,Technical,Un-Organized,2022/TEXT/TSC/07050,Unique,9th Class - Grade 9 with No Experience (OR) Gr...,16,https://s3.ap-south-1.amazonaws.com/nsdcproddo...
2726,2727,Textile,Handloom & Khadi,Spinner,Twister cum Doubler - Handloom,Textile_Twister cum Doubler - Handloom,TSC/Q7902,3.0,NaN,31/03/2025,...,I,Manufacturing,NaN,Technical,Un-Organized,2022/TEXT/TSC/07028,Unique,Ability to read and write,18,https://s3.ap-south-1.amazonaws.com/nsdcproddo...
2727,2728,Textile,Technical Textiles,Spinning - Technical Textiles,Tape Plant Operator,Textile_Tape Plant Operator,TSC/Q8201,3.0,NaN,31/03/2025,...,I,Manufacturing,NaN,Technical,Organized,2022/TEXT/TSC/07079,QP with Electives,9th Class - with No Experience (OR) Grade 8 pa...,18,https://s3.ap-south-1.amazonaws.com/nsdcproddo...
2728,2729,Textile,Technical Textiles,Post Spinning - Technical Textiles,Tape Winder,Textile_Tape Winder,TSC/Q8301,3.0,NaN,31/03/2025,...,I,Manufacturing,NaN,Technical,Organized,2022/TEXT/TSC/07080,Unique,9th Class - with No Experience (OR) Grade 8 pa...,18,https://s3.ap-south-1.amazonaws.com/nsdcproddo...


In [6]:
cd Internship/CEEW/

d:\Internship\CEEW


In [7]:
from tqdm import tqdm
import os
import pandas as pd
import logging
import requests
import PyPDF2
import re

# Define the path for saving PDFs and the output CSV
output_folder = "Jobpdfs"
os.makedirs(output_folder, exist_ok=True)
output_csv_path = os.path.join(output_folder, "final_extracted_job_info.csv")
error_log_path = os.path.join(output_folder, "error_log.csv")

# Setup logging
logging.basicConfig(filename='processing.log', level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

# Function to download and save PDF
def download_pdf(url, save_path):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses
        with open(save_path, 'wb') as file:
            file.write(response.content)
        logging.info(f"Downloaded PDF from {url} to {save_path}")
    except Exception as e:
        logging.error(f"Error downloading PDF from {url}: {e}")
        raise

# Function to extract text from the third page of a PDF file
def extract_from_file(pdf_path):
    try:
        with open(pdf_path, "rb") as file:
            pdf_reader = PyPDF2.PdfReader(file)
            if len(pdf_reader.pages) < 3:
                logging.warning(f"PDF {pdf_path} has fewer than 3 pages.")
                return "Error: PDF has fewer than 3 pages."
            # Extract text from the third page (0-based index 2)
            page_text = pdf_reader.pages[2].extract_text()
            if page_text is None:
                logging.warning(f"PDF {pdf_path} has no text on the third page.")
                return "Error: No text on the third page."
            return page_text.replace('\n', ' ')
    except Exception as e:
        logging.error(f"Error extracting text from {pdf_path}: {e}")
        return f"Error: {str(e)}"

# Function to extract job info from text
def extract_job_info(text):
    # Define regex pattern for job title and brief job description
    pattern = re.compile(r'([A-Z]{2,}/Q\d{4}:\s*.+?)\s+Brief Job Description\s+(.+?)\s+Personal Attributes', re.DOTALL)
    matches = pattern.findall(text)
    
    job_info = []
    for match in matches:
        job_title = match[0].strip()
        job_description = match[1].strip()
        job_info.append(f"Job Title: {job_title}\nJob Description: {job_description}\nPerformance Criteria: ")
    
    if not job_info:  # If no matches found, consider it an error in extraction
        job_info.append(f"Error: No valid job information extracted from the text.")
    
    return job_info

def process_pdfs(filtered_df):
    all_jobs_info = []
    errors = []

    for idx, row in tqdm(filtered_df.iterrows(), total=filtered_df.shape[0], desc="Processing PDFs"):
        serial_no = row.get('S No.', 'N/A')
        url = row['Link']
        file_name = f"{row['QP/Job Role Name'].replace(' ', '_')}.pdf"
        pdf_path = os.path.join(output_folder, file_name)

        try:
            # Download and save the PDF
            download_pdf(url, pdf_path)

            # Extract text from the third page
            qp_text = extract_from_file(pdf_path)

            # Extract job information from the text
            job_info = extract_job_info(qp_text)
            combined_info = '\n'.join(job_info)
            all_jobs_info.append({
                'S No.': serial_no,
                'Job Info': combined_info
            })
            # Print the extracted information
            print(f"Extracted Information for {file_name}:")
            print(combined_info)
            print("=" * 80)
        except Exception as e:
            logging.error(f"Error processing URL {url} for serial number {serial_no}: {e}")
            error_message = f"Error: {str(e)}"
            all_jobs_info.append({
                'S No.': serial_no,
                'Job Info': error_message
            })
            errors.append({'Serial No': serial_no, 'Error': error_message})

    # Save all extracted job info to a single CSV file
    if all_jobs_info:
        job_df = pd.DataFrame(all_jobs_info)
        job_df.to_csv(output_csv_path, index=False, encoding="utf-8")
        print(f"Final extracted job information saved to {output_csv_path}")

    # Save errors to a CSV file
    if errors:
        errors_df = pd.DataFrame(errors)
        errors_df.to_csv(error_log_path, index=False, encoding="utf-8")
        print(f"Errors logged to {error_log_path}")

# Process the PDFs and save the extracted information to a CSV file
process_pdfs(filtered_df)


Processing PDFs:   0%|          | 0/838 [00:00<?, ?it/s]

Processing PDFs:   0%|          | 1/838 [00:01<27:21,  1.96s/it]

Extracted Information for Automotive_Sales_Executive.pdf:
Job Title: ASC/Q1001: Automotive Sales Executive
Job Description: The individual primarily handles potential customer leads, presents value propositions for vehicles and manages vehicle retail sales.
Performance Criteria: 


Processing PDFs:   0%|          | 2/838 [00:03<26:34,  1.91s/it]

Extracted Information for Automotive_Sales_Executive.pdf:
Job Title: ASC/Q1001: Automotive Sales Executive
Job Description: The individual primarily handles potential customer leads, presents value propositions for vehicles and manages vehicle retail sales.
Performance Criteria: 


...

Extracted Information for Tape_Plant_Operator.pdf:
Job Title: TSC/Q8201: Tape Plant Operator
Job Description: The Tape plant operator is responsible for running the Tape plant under supervision by maintaining health, safety and security in the raﬃa industry.
Performance Criteria: 


Processing PDFs: 100%|█████████▉| 837/838 [22:55<00:01,  1.54s/it]

Extracted Information for Tape_Winder.pdf:
Job Title: TSC/Q8301: Tape Winder
Job Description: The Tape winder man is responsible for running the Tape winding machine under supervision by maintaining health, safety and security in the raﬃa industry.
Performance Criteria: 


Processing PDFs: 100%|██████████| 838/838 [22:56<00:00,  1.64s/it]

Extracted Information for Circular_loom_operator_(Shuttle_type).pdf:
Job Title: TSC/Q8401: Circular loom operator (Shuttle type)
Job Description: The Circular loom operator (Shuttle type) is responsible for running the Circular loom (Shuttle type) under supervision by maintaining health, safety and security in the raﬃa industry.
Performance Criteria: 
Final extracted job information saved to Jobpdfs\final_extracted_job_info.csv
Errors logged to Jobpdfs\error_log.csv


In [22]:
import os
import pandas as pd
import logging
import requests
import PyPDF2
import re

# Define the path for saving PDFs, checkpoints, and the output CSV
output_folder = "Jobpdfs_problematic"
os.makedirs(output_folder, exist_ok=True)
output_csv_path = os.path.join(output_folder, "final_extracted_job_info_problematic.csv")

# Setup logging
logging.basicConfig(filename='processing_problematic.log', level=logging.DEBUG)

# Function to download and save PDF
def download_pdf(url, save_path):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses
        with open(save_path, 'wb') as file:
            file.write(response.content)
        logging.info(f"Downloaded PDF from {url} to {save_path}")
    except Exception as e:
        logging.error(f"Error downloading PDF from {url}: {e}")

# Function to extract text from the third page of a PDF file
def extract_from_file(pdf_path):
    try:
        with open(pdf_path, "rb") as file:
            pdf_reader = PyPDF2.PdfReader(file)
            if len(pdf_reader.pages) < 3:
                logging.warning(f"PDF {pdf_path} has fewer than 3 pages.")
                return ""
            # Extract text from the third page (0-based index 2)
            page_text = pdf_reader.pages[2].extract_text()
            return page_text.replace('\n', ' ')
    except Exception as e:
        logging.error(f"Error extracting text from {pdf_path}: {e}")
        return ""

# Function to extract job info from text
def extract_job_info(text):
    # Define regex pattern for job title and brief job description
    pattern = re.compile(r'([A-Z]{2,}/Q\d{4}:\s*.+?)\s+Brief Job Description\s+(.+?)\s+Personal Attributes', re.DOTALL)
    matches = pattern.findall(text)
    
    job_info = []
    for match in matches:
        job_title = match[0].strip()
        job_description = match[1].strip()
        job_info.append(f"Job Title: {job_title}\nJob Description: {job_description}\nPerformance Criteria: ")
    
    return job_info

# Identify rows where 'QP/Job Role Name' contains a '/'
problematic_entries = filtered_df[filtered_df['QP/Job Role Name'].str.contains('/', na=False)]

# List to hold all extracted job information
all_job_info = []

# Iterate over the problematic entries and process them
for idx, row in problematic_entries.iterrows():
    try:
        url = row['Link']
        # Sanitize the file name by replacing '/' with '_' for saving only
        sanitized_name = row['QP/Job Role Name'].replace('/', '_')
        file_name = f"{sanitized_name}.pdf"
        pdf_path = os.path.join(output_folder, file_name)

        # Download and save the PDF
        download_pdf(url, pdf_path)

        # Extract text from the third page
        qp_text = extract_from_file(pdf_path)

        # Extract job information from the text
        job_info = extract_job_info(qp_text)
        if job_info:
            # Append each job info entry with the original 'QP/Job Role Name' containing '/'
            for info in job_info:
                all_job_info.append({'Index': idx, 'S No.': row['S No.'], 'Job Info': info, 'QP/Job Role Name': row['QP/Job Role Name']})
            # Print the extracted information
            print(f"Extracted Information for {file_name}:")
            print('\n'.join(job_info))
            print("=" * 80)
    except Exception as e:
        logging.error(f"Error processing URL {row['Link']}: {e}")
        print(f"Error processing URL {row['Link']}: {e}")

# Save all extracted job information to the final output CSV
if all_job_info:
    final_df = pd.DataFrame(all_job_info)
    final_df.to_csv(output_csv_path, index=False, encoding="utf-8")
    print(f"Final extracted job information saved to {output_csv_path}")
    
    # Print the first 5 entries of the final DataFrame
    print("\nFirst 5 entries of the final extracted job information:")
    print(final_df.head())
else:
    print("No job information extracted.")


Extracted Information for Test Engineer-Product_Vehicle.pdf:
Job Title: ASC/Q8403: Test Engineer-Product/Vehicle
Job Description: This role is responsible for conducting various indoor tests on automobile aggregates like engines, transmission system, braking system, fuel handling system, electrical and electronic components, body frames etc. as well as testing of built up vehicle prototypes in a simulated laboratory environment and on road setup, ensuring proper setup of test apparatus, documenting test results and presenting ﬁrst level analysis report to the test manager.
Performance Criteria: 
Extracted Information for Forklift Operator_Driver.pdf:
Job Title: ASC/Q9707: Forklift Operator/Driver
Job Description: The individual is responsible for ensuring the ﬁtness of the forklift for use, driving the forklift, handling delivery, stocking, and shifting of the goods, maintaining safe and healthy working practices and communicating eﬀectively with colleagues, customers, and others.
Perf

PermissionError: [Errno 13] Permission denied: 'Jobpdfs_problematic\\final_extracted_job_info_problematic.csv'

In [24]:
import re

# Define the path to the log file and the output file for the analysis
log_file_path = 'D:\\processing.log'
errors_output_path = 'errors_summary.txt'

def analyze_logs(log_file_path, errors_output_path):
    errors_summary = []

    with open(log_file_path, 'r') as log_file:
        lines = log_file.readlines()
    
    # Define regex patterns to match different types of errors
    request_error_pattern = re.compile(r"Request error downloading PDF from (.+?): (.+)")
    save_error_pattern = re.compile(r"Error saving PDF from (.+?) to (.+?): (.+)")
    pdf_read_error_pattern = re.compile(r"PDF read error from (.+?): (.+)")
    extraction_error_pattern = re.compile(r"Error extracting text from (.+?): (.+)")
    warning_pattern = re.compile(r"PDF (.+?) has fewer than 3 pages\.|PDF (.+?) has no text on the third page\.")
    
    for line in lines:
        # Match request errors
        request_match = request_error_pattern.search(line)
        if request_match:
            url, error_message = request_match.groups()
            errors_summary.append(f"Request Error: URL: {url}, Message: {error_message}")
            continue
        
        # Match save errors
        save_match = save_error_pattern.search(line)
        if save_match:
            url, path, error_message = save_match.groups()
            errors_summary.append(f"Save Error: URL: {url}, Path: {path}, Message: {error_message}")
            continue
        
        # Match PDF read errors
        pdf_read_match = pdf_read_error_pattern.search(line)
        if pdf_read_match:
            path, error_message = pdf_read_match.groups()
            errors_summary.append(f"PDF Read Error: Path: {path}, Message: {error_message}")
            continue
        
        # Match extraction errors
        extraction_match = extraction_error_pattern.search(line)
        if extraction_match:
            path, error_message = extraction_match.groups()
            errors_summary.append(f"Extraction Error: Path: {path}, Message: {error_message}")
            continue
        
        # Match warnings
        warning_match = warning_pattern.search(line)
        if warning_match:
            path = warning_match.group(1) or warning_match.group(2)
            errors_summary.append(f"Warning: Path: {path}")

    # Write the errors summary to a text file
    with open(errors_output_path, 'w') as output_file:
        for error in errors_summary:
            output_file.write(f"{error}\n")
    
    print(f"Errors summary saved to {errors_output_path}")

# Analyze the logs and save the summary
analyze_logs(log_file_path, errors_output_path)


Errors summary saved to errors_summary.txt


In [2]:
!py -m pip install unidecode

   ---------------------------------------- 0.0/235.5 kB ? eta -:--:--
   ----- --------------------------------- 30.7/235.5 kB 660.6 kB/s eta 0:00:01
   ---------- ---------------------------- 61.4/235.5 kB 656.4 kB/s eta 0:00:01
   --------------- ----------------------- 92.2/235.5 kB 655.4 kB/s eta 0:00:01
   ------------------- ------------------ 122.9/235.5 kB 654.9 kB/s eta 0:00:01
   ------------------------ ------------- 153.6/235.5 kB 654.6 kB/s eta 0:00:01
   --------------------------------- ---- 204.8/235.5 kB 692.4 kB/s eta 0:00:01
   --------------------------------- ---- 204.8/235.5 kB 692.4 kB/s eta 0:00:01
   -------------------------------------- 235.5/235.5 kB 654.8 kB/s eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.3.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import re
from unidecode import unidecode

def clean_text(text):
    # Normalize text to remove any special characters and convert to ASCII
    text = unidecode(text)
    
    # Specific replacements for known encoding issues
    replacements = {
        "i!EUR": "o",  # Assuming "i!EUR" should be "o"
        "i!,ow": "flow",  # Assuming "i!,ow" should be "flow"
        "doi!EUR": "doff",  # Assuming "doi!EUR" should be "doff"
    }
    
    for original, replacement in replacements.items():
        text = text.replace(original, replacement)
    
    # Remove unwanted characters (like ï¬€) and normalize spaces
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Example text
job_description = ("A Two For One Twister (TFO) tenter is responsible for carrying out tenting activities and ensuring minimum machine stoppage and achieving maximum production in TFO machine. "
                   "He should be able to creel the paralleled yarn, piece the yarn on breakage, doi!EUR the doubled cone package, carry out cleaning and preventive maintenance activities thus producing defect free doubled yarn. "
                   "This job requires the individual to have thorough knowledge in process i!,ow and material i!,ow in a textile mill for yarn production and should be able to know the important functions and operations of a TFO machine")

# Clean the text
cleaned_job_description = clean_text(job_description)

# Print cleaned text
print(f"Job Description: {cleaned_job_description}")


Job Description: A Two For One Twister (TFO) tenter is responsible for carrying out tenting activities and ensuring minimum machine stoppage and achieving maximum production in TFO machine. He should be able to creel the paralleled yarn, piece the yarn on breakage, doo the doubled cone package, carry out cleaning and preventive maintenance activities thus producing defect free doubled yarn. This job requires the individual to have thorough knowledge in process flow and material flow in a textile mill for yarn production and should be able to know the important functions and operations of a TFO machine


In [19]:
import pandas as pd
text_df=pd.read_csv("D:\\commondata.csv")

In [29]:
text_df.to_csv("D:\\Internship\\CEEW\\descriptionjobs.csv")

PermissionError: [Errno 13] Permission denied: 'D:\\Internship\\CEEW\\descriptionjobs.csv'

In [ ]:
import pandas as pd
import re
from unidecode import unidecode
from textblob import TextBlob
from tqdm import tqdm  # Import tqdm for the progress bar

# Function to clean and correct text
def clean_and_correct_text(text):
    # Normalize text to remove any special characters and convert to ASCII
    text = unidecode(text)
    
    # Specific replacements for known encoding issues
    replacements = {
        "i!EUR": "o",  # Assuming "i!EUR" should be "o"
        "i!,ow": "flow",  # Assuming "i!,ow" should be "flow"
        "doi!EUR": "doff",  # Assuming "doi!EUR" should be "doff"
        "ï¬‚ow": "flow",  # Assuming "ï¬‚ow" should be "flow"
        "aï¬ƒliated": "affiliated",  # Assuming "aï¬ƒliated" should be "affiliated"
        "identiï¬es": "identifies",  # Assuming "identiï¬es" should be "identifies"
        "ï¬nal": "final"  # Assuming "ï¬nal" should be "final"
    }
    
    for original, replacement in replacements.items():
        text = text.replace(original, replacement)
    
    # Remove unwanted characters and normalize spaces
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    # List of words to be excluded from spell-check correction
    exclude_from_correction = ["Job", "handles", "HEMM", "diagnoses", "bulges", "requirements"]

    # Spell check and correct the text
    corrected_text = []
    for word in text.split():
        if word in exclude_from_correction:
            corrected_text.append(word)
        else:
            corrected_word = str(TextBlob(word).correct())
            corrected_text.append(corrected_word)
    
    text = ' '.join(corrected_text)
    
    return text

# Assuming you have your DataFrame already loaded as text_df
# Apply the clean_and_correct_text function to the final_pc column with tqdm
tqdm.pandas()  # Initialize tqdm for pandas
text_df['final_pc'] = text_df['final_pc'].progress_apply(clean_and_correct_text)

# Display the cleaned and corrected DataFrame
print(text_df)


  0%|          | 0/838 [00:00<?, ?it/s]

100%|██████████| 838/838 [16:59<00:00,  1.22s/it]

     Unnamed: 0  S No.      Sector                         Sub Sector  \
0           249    250  Automotive  Automotive Vehicle Sales (Dealer)   
1           250    251  Automotive  Automotive Vehicle Sales (Dealer)   
2           251    252  Automotive  Automotive Vehicle Sales (Dealer)   
3           252    253  Automotive  Automotive Vehicle Sales (Dealer)   
4           253    254  Automotive  Automotive Vehicle Sales (Dealer)   
..          ...    ...         ...                                ...   
833        2725   2726     Textile                   Handloom & Khadi   
834        2726   2727     Textile                   Handloom & Khadi   
835        2727   2728     Textile                 Technical Textiles   
836        2728   2729     Textile                 Technical Textiles   
837        2729   2730     Textile                 Technical Textiles   

                             Occupation  \
0                         Vehicle Sales   
1                         Vehicle Sal

In [ ]:
text_df.to_csv("D:\\Internship\\CEEW\\descriptionjobs.csv")

In [28]:
# Assuming text_df is your DataFrame

# Convert 'Version' to a numeric type if it's not already
text_df['Version'] = pd.to_numeric(text_df['Version'], errors='coerce')

# Sort the DataFrame by 'QP/Job Role Name', 'NSQF Level', and 'Version' (descending order)
text_df = text_df.sort_values(by=['QP/Job Role Name', 'NSQF Level', 'Version'], ascending=[True, True, False])

# Drop duplicates, keeping only the first entry of each combination of 'QP/Job Role Name' and 'NSQF Level'
text_df_latest = text_df.drop_duplicates(subset=['QP/Job Role Name', 'NSQF Level'], keep='first')

# text_df_latest will now contain only the latest versions


In [30]:
text_df_latest

,Unnamed: 0,S No.,Sector,Sub Sector,Occupation,QP/Job Role Name,Unnamed: 5,QP/Job Role Code,Version,Last Reviewed On(DD/MM/YYYY),...,Manufacturing/Services/Agriculture,Deactivation Date(DD/MM/YYYY),Technical/Non Technical,Organised Or Unorganised Or Both,NQR code,QP Type,Minimum Education Qualification & Experience,Minimum Job Entry Age,Link,final_pc
377,1148,1149,Handicrafts and Carpet,Glassware,"Decoration (Etching, Pasting & Painting) and F...",Abrasion & Grinding Machine Operator,Handicrafts and Carpet_Abrasion & Grinding Mac...,HCS/Q2304,2,NaN,...,Manufacturing,NaN,Technical,Both,QG-03-HC-00583-2023-V2-HCSSC,Unique,"10th grade pass,8th grade pass and pursuing co...",18.0,https://s3.ap-south-1.amazonaws.com/nsdcproddo...,Job Title: HCS/Q2304: Abrasion & Grinding Mach...
388,1159,1160,Handicrafts and Carpet,Metalware,Cleaning/Polishing/Buffing-Metalware,Acid Cleaner (Metalware),Handicrafts and Carpet_Acid Cleaner (Metalware),HCS/Q3001,2,NaN,...,Manufacturing,NaN,Technical,Both,QG-03-HC-00584-2023-V2-HCSSC,Unique,"10th grade pass,9th grade pass and 1 Year Expe...",18.0,https://s3.ap-south-1.amazonaws.com/nsdcproddo...,Job Title: HCS/Q3001: Acid Cleaner (Metalware)...
184,433,434,Automotive,Manufacturing and R&D,Production Engineering,Additive Manufacturing,Automotive_Additive Manufacturing,ASC/Q6420,1,NaN,...,Manufacturing,NaN,Technical,Organized,QG-4.5-AU-01822-2024-V1-ASDC,Unique,NaN,25.0,https://s3.ap-south-1.amazonaws.com/nsdcproddo...,Job Title: ASC/Q6420: Additive Manufacturing J...
347,1118,1119,Handicrafts and Carpet,Hand Crafted Textiles,Hand Block Printing,Advanced Program on Hand Block Printing,Handicrafts and Carpet_Advanced Program on Han...,HCS/N9204,1,NaN,...,Services,NaN,Technical,Both,NG-04-HC-00632-2023-V1-HCSSC,NaN,"11th grade pass,10th grade pass and pursuing c...",NaN,https://s3.ap-south-1.amazonaws.com/nsdcproddo...,Job Title: HCS/N9204: Advanced Program on And ...
348,1119,1120,Handicrafts and Carpet,Hand Crafted Textiles,"Hand Embroidery, Designing and Finishing",Advanced Program on Traditional Hand Embroidery,Handicrafts and Carpet_Advanced Program on Tra...,HCS/N9205,1,NaN,...,Services,NaN,Technical,Both,NG-04-HC-00635-2023-V1-HCSSC,NaN,"11th grade pass,10th grade pass and pursuing c...",NaN,https://s3.ap-south-1.amazonaws.com/nsdcproddo...,Job Title: HCS/N9205: Advanced Program on Trad...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,2272,2273,Green Jobs,Renewable Energy,Site survey,Wind Resource Assessor and Site Surveyor - Win...,Green Jobs_Wind Resource Assessor and Site Sur...,SGJ/Q1202,3,NaN,...,Services,NaN,Technical,Organized,QG-05-PW-00530-2023-V1.1-SCGJ,Unique,UG in relevant field - completed second year o...,18.0,https://s3.ap-south-1.amazonaws.com/nsdcproddo...,Job Title: SGJ/Q1202: Mind Resource Assessor a...
543,1900,1901,Mining,Mining Operation,Loading and Hauling - Underground,Winding Operator,Mining_Winding Operator,MIN/Q1503,2,NaN,...,Manufacturing,NaN,Technical,Organized,2022/MIN/SCMS/06975,QP with Electives,"12th Class - or its equivalent, with 2 years r...",20.0,https://s3.ap-south-1.amazonaws.com/nsdcproddo...,Job Title: MIN/Q1503: Finding Operator Job Des...
783,2675,2676,Textile,Spinning – Textiles,Post Spinning Operations,Winding Tenter,Textile_Winding Tenter,TSC/Q0305,2,NaN,...,Manufacturing,NaN,Technical,Organized,2022/TEXT/TSC/07054,QP with Electives,9th Class - with No Experience (OR) Grade 8 pa...,18.0,https://s3.ap-south-1.amazonaws.com/nsdcproddo...,Job Title: TSC/Q0305: Finding Enter Job Descri...
532,1889,1890,Mining,Mining Operation,Drilling/Cutting,Wire Saw Operator,Mining_Wire Saw Operator,MIN/Q1201,2,NaN,...,Manufacturing,NaN,Technical,Both,2022/MIN/SCMS/06979,QP with Electives,8th grade pass plus 2-year NTC plus 1 Year NAC,20.0,https://s3.ap-south-1.amazonaws.com/nsdcproddo...,Job Title: MIN/Q1201: Fire Saw Operator Job De...


In [31]:
merged_df_job=pd.read_csv("D:\Internship\CEEW\merged_df_job.csv")

In [42]:
# Perform a left merge to ensure all rows from merged_df_job are retained
merged_result = merged_df_job.merge(
    text_df_latest[['Sector', 'QP/Job Role Name', 'Version', 'NSQF Level', 'final_pc']],
    on=['Sector', 'QP/Job Role Name', 'Version', 'NSQF Level'],
    how='left',
    suffixes=('_merged', '_latest')  # Suffix to indicate the source of final_pc
)

# Concatenate the final_pc columns, placing the one from text_df_latest first
merged_result['final_pc'] = merged_result['final_pc_latest'].fillna('') + merged_result['final_pc_merged'].fillna('')

# Drop the intermediate final_pc columns
merged_result = merged_result.drop(columns=['final_pc_latest', 'final_pc_merged'])

# merged_result now contains a single final_pc column with the values concatenated in the correct order


In [45]:
merged_result.to_csv('D:\\Internship\\CEEW\\merged_df_job_context.csv')